# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Send HTTP request to the target Website

In [4]:
first_page_response = requests.get("https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/1/")

check the response status code

In [5]:
first_page_response.status_code

200

### Parse the response content with BeautifulSoap

In [6]:
soup = BeautifulSoup(first_page_response.text, "html.parser")

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [7]:
first_page_products = soup.select('.product-grid-item')

In [8]:
fp_parsed_products = []
for p in first_page_products:
    title = p.select_one(".wd-entities-title").text.strip()
    discount = p.select_one('.onsale')
    reg_price = p.select_one('.woocommerce-Price-amount')
    reg_price = reg_price.find("bdi").get_text(strip=True)
    if discount is not None:
        discount_price = p.select_one("ins .woocommerce-Price-amount")
        discount_price = discount_price.find("bdi").get_text(strip=True)
    else:
        discount_price = "Nema"
    product_url = p.select_one('.wd-entities-title a').get("href")
    btn = p.select_one('.wd-add-btn a').get("href")

    fp_products_dictionary = {
        "Title": title,
        "RegularPrice": reg_price,
        "DiscountPrice": discount_price,
        "ProductURL": product_url,
        "AddToCart": btn
    }
    fp_parsed_products.append(fp_products_dictionary)

In [9]:
fp_parsed_products

[{'Title': 'Acer A315-23-A7KD',
  'RegularPrice': '17.590ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/acer-a315-23-a7kd/',
  'AddToCart': '?add-to-cart=21494'},
 {'Title': 'Acer A315-23-R5P2',
  'RegularPrice': '27.490ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/acer-a315-23-r5p2/',
  'AddToCart': '?add-to-cart=21510'},
 {'Title': 'ACER Aspire 1 A115-22',
  'RegularPrice': '18.999ден',
  'DiscountPrice': '15.999ден',
  'ProductURL': 'https://clevershop.mk/product/acer-aspire-1-nx-a7pex-001/',
  'AddToCart': '?add-to-cart=20826'},
 {'Title': 'Acer Aspire 3 A315-23-R26A',
  'RegularPrice': '29.990ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/acer-aspire-3-a315-23-r26a/',
  'AddToCart': '?add-to-cart=21516'},
 {'Title': 'Acer Aspire 3 A315-58-33WK',
  'RegularPrice': '24.490ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/21498/',
  'AddToCart': '?add-to-c

Repeat the extraction process for each page of products

In [14]:
base_url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/"
page = 1
parsed_products = []
while True:
    url = base_url + str(page) + '/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    products = soup.select('.product-grid-item')
    if not products:
        break
    for p in products:
        title = p.select_one(".wd-entities-title").text.strip()
        discount = p.select_one('.onsale')
        reg_price = p.select_one('.woocommerce-Price-amount')
        if reg_price is not None:
            reg_price = reg_price.find("bdi").get_text(strip=True)
        else:
            reg_price = "Nema"
        if discount is not None:
            discount_price = p.select_one("ins .woocommerce-Price-amount")
            if discount_price is not None:
                discount_price = discount_price.find("bdi").get_text(strip=True)
            else:
                discount_price = "Nema"
        else:
            discount_price = "Nema"
        product_url = p.select_one('.wd-entities-title a').get("href")
        btn = p.select_one('.wd-add-btn a').get("href")
    
        products_dictionary = {
            "Title": title,
            "RegularPrice": reg_price,
            "DiscountPrice": discount_price,
            "ProductURL": product_url,
            "AddToCart": btn
        }
        parsed_products.append(products_dictionary)

    page += 1

In [15]:
parsed_products

[{'Title': 'Acer A315-23-A7KD',
  'RegularPrice': '17.590ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/acer-a315-23-a7kd/',
  'AddToCart': '?add-to-cart=21494'},
 {'Title': 'Acer A315-23-R5P2',
  'RegularPrice': '27.490ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/acer-a315-23-r5p2/',
  'AddToCart': '?add-to-cart=21510'},
 {'Title': 'ACER Aspire 1 A115-22',
  'RegularPrice': '18.999ден',
  'DiscountPrice': '15.999ден',
  'ProductURL': 'https://clevershop.mk/product/acer-aspire-1-nx-a7pex-001/',
  'AddToCart': '?add-to-cart=20826'},
 {'Title': 'Acer Aspire 3 A315-23-R26A',
  'RegularPrice': '29.990ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/acer-aspire-3-a315-23-r26a/',
  'AddToCart': '?add-to-cart=21516'},
 {'Title': 'Acer Aspire 3 A315-58-33WK',
  'RegularPrice': '24.490ден',
  'DiscountPrice': 'Nema',
  'ProductURL': 'https://clevershop.mk/product/21498/',
  'AddToCart': '?add-to-c

### Create a pandas DataFrame with the scraped products

In [16]:
df = pd.DataFrame(parsed_products)
df

,Title,RegularPrice,DiscountPrice,ProductURL,AddToCart
0,Acer A315-23-A7KD,17.590ден,Nema,https://clevershop.mk/product/acer-a315-23-a7kd/,?add-to-cart=21494
1,Acer A315-23-R5P2,27.490ден,Nema,https://clevershop.mk/product/acer-a315-23-r5p2/,?add-to-cart=21510
2,ACER Aspire 1 A115-22,18.999ден,15.999ден,https://clevershop.mk/product/acer-aspire-1-nx...,?add-to-cart=20826
3,Acer Aspire 3 A315-23-R26A,29.990ден,Nema,https://clevershop.mk/product/acer-aspire-3-a3...,?add-to-cart=21516
4,Acer Aspire 3 A315-58-33WK,24.490ден,Nema,https://clevershop.mk/product/21498/,?add-to-cart=21498
...,...,...,...,...,...
315,Monitor 27 Philips 272E1GAJ/00 VA 1ms 144Hz,12.890ден,Nema,https://clevershop.mk/product/monitor-27-phili...,?add-to-cart=12618
316,Philips 24″ 243V7QDSB,8.390ден,Nema,https://clevershop.mk/product/philips-24%e2%80...,?add-to-cart=12396
317,Philips 27″ 278E1A/00 4K UHD IPS,18.990ден,Nema,https://clevershop.mk/product/hp-27%e2%80%b3-2...,?add-to-cart=12218
318,Philips 279C9-00 MON LED 27″ 3840 x 2160 5Ms 6...,26.990ден,Nema,https://clevershop.mk/product/philips-279c9-00...,?add-to-cart=12578


Save the dataframe as `.csv`

In [23]:
df.to_csv('VNP_Lab1_223223.csv', index=False)